In [121]:
import os
from os import listdir
from os.path import isfile, join, exists
import glob
import subprocess
import math

In [122]:
###############################################################################
# REMatcher is a utility class to make regex's work similar to Perl
# NOTE: if there are any changes needed in this function contact Sean.
###############################################################################
class REMatcher(object):
    ''' This is a utility function to help out with matching regex's and
        grabbing the matches out in a way that is similar to how perl
        does it.
    '''
    def __init__(self, matchstring: str) -> None:
        self.matchstring = matchstring

    def match(self, regexp: str) -> bool:
        self.rematch = re.match(regexp, self.matchstring,
                                re.IGNORECASE |
                                re.MULTILINE |
                                re.DOTALL
                                )
        return bool(self.rematch)

    def group(self, i: int) -> str:
        return "Error 999" if self.rematch is None else self.rematch.group(i)

In [123]:
# for(stage = 0; stage < STAGES - 1; stage=stage+1) begin : STAGE_GEN
#     for(i = 0; i < (stage == 0 ? C : C >> stage); i=i+2) begin : ADDER_GEN
#         if (i < C - 2**stage) begin
#             math_adder_carry_lookahead #(N) adder_inst (
#                 .i_a(w_intermediate_sums[stage][i][N-1:0]),  
#                 .i_b(w_intermediate_sums[stage][i + 1][N-1:0]),  
#                 .i_c(1'b0),  
#                 .ow_sum(w_intermediate_sums[stage + 1][i/2][N-1:0]),  
#                 .ow_c(w_c[stage][i/2])  // Adjusted indexing
#             );
#         end else if (i == (C - 2**stage) && stage == 0) begin
#             assign w_intermediate_sums[stage + 1][i/2][N+stage:0] = w_intermediate_sums[stage][i][N+stage-1:0];  
#         end
#     end
# end


In [4]:

import math

def generate_dadda_numbers(N):
    dadda_numbers = [2]
    next_number = 2
    multiply_by = 1.5  # Start by multiplying by 1.5

    while True:
        next_number = math.ceil(next_number * multiply_by)
        if next_number > N:
            break
        dadda_numbers.append(next_number)

        # Alternate between 1.5 and 4/3
        multiply_by = 4 / 3 if multiply_by == 1.5 else 1.5
    return dadda_numbers

N = 16  # You can change this to any value up to which you want Dadda numbers
print(generate_dadda_numbers(N))


[2, 3, 4, 6, 8, 12, 16]


In [5]:
def generate_partial_products(N, f):
    # Generate partial products
    f.write("// Partial products generation\n")

    # Define bit groups here, then populate it as we generate the partial products
    bit_groups = {i: [] for i in range(2 * N)}

    # Determine the max number of digits needed
    max_digits = len(str(N - 1))
    
    for i in range(N):
        for j in range(N):
            formatted_i = str(i).zfill(max_digits)
            formatted_j = str(j).zfill(max_digits)

            f.write(f"wire w_pp_{formatted_i}_{formatted_j} = i_multiplier[{i:2}] & i_multiplicand[{j:2}];\n")
            bit_groups[i + j].append(f"w_pp_{formatted_i}_{formatted_j}")

    f.write("\n")
    return bit_groups


In [6]:
N=8
with open('junk.txt', 'w') as f:
    bit_groups = generate_partial_products(N,f)
    print(bit_groups)

{0: ['w_pp_0_0'], 1: ['w_pp_0_1', 'w_pp_1_0'], 2: ['w_pp_0_2', 'w_pp_1_1', 'w_pp_2_0'], 3: ['w_pp_0_3', 'w_pp_1_2', 'w_pp_2_1', 'w_pp_3_0'], 4: ['w_pp_0_4', 'w_pp_1_3', 'w_pp_2_2', 'w_pp_3_1', 'w_pp_4_0'], 5: ['w_pp_0_5', 'w_pp_1_4', 'w_pp_2_3', 'w_pp_3_2', 'w_pp_4_1', 'w_pp_5_0'], 6: ['w_pp_0_6', 'w_pp_1_5', 'w_pp_2_4', 'w_pp_3_3', 'w_pp_4_2', 'w_pp_5_1', 'w_pp_6_0'], 7: ['w_pp_0_7', 'w_pp_1_6', 'w_pp_2_5', 'w_pp_3_4', 'w_pp_4_3', 'w_pp_5_2', 'w_pp_6_1', 'w_pp_7_0'], 8: ['w_pp_1_7', 'w_pp_2_6', 'w_pp_3_5', 'w_pp_4_4', 'w_pp_5_3', 'w_pp_6_2', 'w_pp_7_1'], 9: ['w_pp_2_7', 'w_pp_3_6', 'w_pp_4_5', 'w_pp_5_4', 'w_pp_6_3', 'w_pp_7_2'], 10: ['w_pp_3_7', 'w_pp_4_6', 'w_pp_5_5', 'w_pp_6_4', 'w_pp_7_3'], 11: ['w_pp_4_7', 'w_pp_5_6', 'w_pp_6_5', 'w_pp_7_4'], 12: ['w_pp_5_7', 'w_pp_6_6', 'w_pp_7_5'], 13: ['w_pp_6_7', 'w_pp_7_6'], 14: ['w_pp_7_7'], 15: []}
